In [ ]:
import pandas as pd
import os

In [ ]:
df = pd.read_csv('/Users/carboni/Downloads/Leopard Dec 19 2023.csv')

### Clusters and other keywords

In [ ]:
#keyword = 'Bête'
#df = df[df['cluster'].str.contains(keyword, case=False, na=False)]

In [ ]:
df.head()

## Index city 

### If WKT

In [ ]:
df[['lon', 'lat']] = df['wkt'].str.extract(r'POINT\(([^ ]+) ([^ ]+)\)')

### if not wkt

In [ ]:
city_coords = df.groupby('City').agg({'lat': 'first', 'lon': 'first'}).reset_index()

In [ ]:
city_coords.rename({'City': 'name'}, axis=1, inplace=True)

In [ ]:
city_coords.index = city_coords.index + 1
city_coords.index.name = 'id'

In [ ]:
print(city_coords)

In [ ]:
city_coords.to_csv('od_city_index.csv', index=True)

## Index to OD Matrix

In [ ]:
#df['Year'] = df['date'].str.split('-', n=1, expand=True)[0]

In [ ]:
df['Year'] = pd.to_datetime(df['Date']).dt.year

In [ ]:
df.head()

In [ ]:
city_id_map = city_coords.reset_index().set_index('name')['id'].to_dict()

In [ ]:
df = df.dropna(subset=['City'])

In [ ]:
df['origin'] = df['City'].map(city_id_map).astype(int)
df['dest'] = df['City'].shift(-1).map(city_id_map).fillna(method='ffill').astype(int)
df.iloc[-1, df.columns.get_loc('dest')] = df.iloc[-1]['origin']

### Overall OD Matrix

In [ ]:
overall_od_matrix = df.groupby(['origin', 'dest', 'Year']).size().reset_index(name='count')

In [ ]:
overall_od_matrix['time'] = overall_od_matrix['Year'].astype(str) + '-01-01'
overall_od_matrix.drop('Year', axis=1, inplace=True)

In [ ]:
print(overall_od_matrix)

In [ ]:
overall_od_matrix.to_csv('overall_od_matrix.csv', index=False)